In [1]:
%run ../paths.py
CONST = CONST()

In [354]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()
import ipywidgets
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics

In [287]:
train_data = pd.read_csv(CONST.TRAIN_DATA)
test_data = pd.read_csv(CONST.TEST_DATA)

In [138]:
train_data.head()

,Unnamed: 0,tweet,class
0,12761,Mannnnnn these hoes aint loyal,1
1,25141,have a fabulous weekend beautiful souls #nofil...,0
2,53167,#fathersday #dad @user,0
3,17294,RT @SheswantstheD: these hoes ain't loyal http...,1
4,29359,newhairâ¨ð #lapis#rollen#new#hair#brond#gr...,0


In [139]:
test_data.head()

,Unnamed: 0,tweet,class
0,22583,What I look like ho? I look like YES and you l...,1
1,29484,@user lots of #resistance and #challenges fro...,0
2,34265,what's wrong with ppl nowadays? i was conducti...,0
3,4155,@Montulos Lakers are trash,1
4,37575,will be playing rf simpson in #singinintherain...,0


In [140]:
%run ../src/features/other_features.py

In [141]:
train_data[['neg','pos','neu','compound']]= train_data["tweet"].progress_apply(lambda x: pd.Series(get_vader_sentiment(x)))

In [142]:
train_data['nouns']=train_data["tweet"].progress_apply(lambda x: get_postag_count(x,'noun')) 
train_data['prons']=train_data["tweet"].progress_apply(lambda x: get_postag_count(x,'pron')) 
train_data['verbs']=train_data["tweet"].progress_apply(lambda x: get_postag_count(x,'verb')) 
train_data['adjs']=train_data["tweet"].progress_apply(lambda x: get_postag_count(x,'adj')) 
train_data['advs']=train_data["tweet"].progress_apply(lambda x: get_postag_count(x,'adv')) 
   
    
 

In [158]:
train_data[['num_exclamation_marks', 'num_question_marks', 'num_symbols', 'stopword_count','emoji_count']] = train_data["tweet"].progress_apply(lambda x: pd.Series(get_text_features(x)))

In [159]:
%run ../src/processing/cleaning.py

In [160]:
train_data["tweet"] = train_data["tweet"].progress_apply(lambda x : clean_text(x))

In [165]:
drop_rows = train_data[train_data['tweet'] ==""]
train_data.drop(drop_rows.index, axis=0,inplace=True)
train_data.replace(to_replace='None', value=np.nan).dropna(inplace=True)
train_data.dropna(inplace=True)

In [206]:
train_data.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42019 entries, 0 to 42363
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             42019 non-null  int64  
 1   tweet                  42019 non-null  object 
 2   class                  42019 non-null  int64  
 3   neg                    42019 non-null  float64
 4   pos                    42019 non-null  float64
 5   neu                    42019 non-null  float64
 6   compound               42019 non-null  float64
 7   nouns                  42019 non-null  int64  
 8   prons                  42019 non-null  int64  
 9   verbs                  42019 non-null  int64  
 10  adjs                   42019 non-null  int64  
 11  advs                   42019 non-null  int64  
 12  num_exclamation_marks  42019 non-null  int64  
 13  num_question_marks     42019 non-null  int64  
 14  num_symbols            42019 non-null  int64  
 15  st

In [163]:
train_data[['tweet','hashtag','user','allcaps']] = train_data["tweet"].progress_apply(lambda x: pd.Series(get_twitter_features(x)))

In [167]:
train_data[['total_length','num_words','num_unique_words','words_vs_unique']] = train_data["tweet"].progress_apply(lambda x: pd.Series(other_text_features(x)))

In [168]:
train_data.head()

,Unnamed: 0,tweet,class,neg,pos,neu,compound,nouns,prons,verbs,...,num_symbols,stopword_count,emoji_count,hashtag,user,allcaps,total_length,num_words,num_unique_words,words_vs_unique
0,12761,mann these hoes not loyal,1,0.390,0.000,0.610,-0.3724,3,0,1,...,0,0,0,0,0,0,25.0,5.0,5.0,1.0
1,25141,fabulous weekend beautiful souls,0,0.000,0.549,0.451,0.8074,3,0,1,...,0,2,0,1,0,0,32.0,4.0,4.0,1.0
3,17294,these hoes not loyal,1,0.299,0.000,0.701,-0.3724,4,0,1,...,0,1,0,0,1,0,20.0,4.0,4.0,1.0
4,29359,newhair,0,0.000,0.000,1.000,0.0000,2,0,1,...,0,0,0,13,0,0,7.0,1.0,1.0,1.0
5,22086,not trust these hoes,1,0.278,0.000,0.722,-0.4023,2,0,1,...,0,2,0,0,0,0,20.0,4.0,4.0,1.0


In [203]:
train_data[train_data["num_words"] >3].tweet

0                                mann these hoes not loyal
1                         fabulous weekend beautiful souls
3                                     these hoes not loyal
5                                     not trust these hoes
6                              these bitches need guidance
                               ...                        
42359      going sell iphone both macs not support faggots
42360                          latest seattle daily thanks
42361    bad enough not being able see braydon weeks ti...
42362                 hardcore british queer anal pounding
42363    thought justin bieber could not anymore pussy ...
Name: tweet, Length: 32196, dtype: object

In [205]:
train_data.to_csv("../dataset/cleaned_data")

In [288]:
test_data[['neg','pos','neu','compound']]= test_data["tweet"].progress_apply(lambda x: pd.Series(get_vader_sentiment(x)))

In [289]:
test_data['nouns']=test_data["tweet"].progress_apply(lambda x: get_postag_count(x,'noun')) 
test_data['prons']=test_data["tweet"].progress_apply(lambda x: get_postag_count(x,'pron')) 
test_data['verbs']=test_data["tweet"].progress_apply(lambda x: get_postag_count(x,'verb')) 
test_data['adjs']=test_data["tweet"].progress_apply(lambda x: get_postag_count(x,'adj')) 
test_data['advs']=test_data["tweet"].progress_apply(lambda x: get_postag_count(x,'adv')) 
   
    
 

In [290]:
test_data[['num_exclamation_marks', 'num_question_marks', 'num_symbols', 'stopword_count','emoji_count']] = test_data["tweet"].progress_apply(lambda x: pd.Series(get_text_features(x)))

In [291]:
test_data["tweet"] = test_data["tweet"].progress_apply(lambda x : clean_text(x))

In [295]:
drop_rows = test_data[test_data['tweet'] ==""]
test_data.drop(drop_rows.index, axis=0,inplace=True)
test_data.replace(to_replace='None', value=np.nan).dropna(inplace=True)
test_data.dropna(inplace=True)

In [296]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11866 entries, 0 to 11948
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             11866 non-null  int64  
 1   tweet                  11866 non-null  object 
 2   class                  11866 non-null  int64  
 3   neg                    11866 non-null  float64
 4   pos                    11866 non-null  float64
 5   neu                    11866 non-null  float64
 6   compound               11866 non-null  float64
 7   nouns                  11866 non-null  int64  
 8   prons                  11866 non-null  int64  
 9   verbs                  11866 non-null  int64  
 10  adjs                   11866 non-null  int64  
 11  advs                   11866 non-null  int64  
 12  num_exclamation_marks  11866 non-null  int64  
 13  num_question_marks     11866 non-null  int64  
 14  num_symbols            11866 non-null  int64  
 15  st

In [294]:
test_data[['tweet','hashtag','user','allcaps']] = test_data["tweet"].progress_apply(lambda x: pd.Series(get_twitter_features(x)))

In [297]:
test_data[['total_length','num_words','num_unique_words','words_vs_unique']] = test_data["tweet"].progress_apply(lambda x: pd.Series(other_text_features(x)))

In [226]:
test_data.tweet[11948]

'bitch work hours protect freedom fuck'

In [301]:
train_data = pd.read_csv("../dataset/cleaned_data")

In [308]:
train_data.head()

,tweet,class,neg,pos,neu,compound,nouns,prons,verbs,adjs,...,num_symbols,stopword_count,emoji_count,hashtag,user,allcaps,total_length,num_words,num_unique_words,words_vs_unique
0,mann these hoes not loyal,1,0.390,0.000,0.610,-0.3724,3,0,1,0,...,0,0,0,0,0,0,25.0,5.0,5.0,1.0
1,fabulous weekend beautiful souls,0,0.000,0.549,0.451,0.8074,3,0,1,2,...,0,2,0,1,0,0,32.0,4.0,4.0,1.0
2,these hoes not loyal,1,0.299,0.000,0.701,-0.3724,4,0,1,1,...,0,1,0,0,1,0,20.0,4.0,4.0,1.0
3,newhair,0,0.000,0.000,1.000,0.0000,2,0,1,0,...,0,0,0,13,0,0,7.0,1.0,1.0,1.0
4,not trust these hoes,1,0.278,0.000,0.722,-0.4023,2,0,1,1,...,0,2,0,0,0,0,20.0,4.0,4.0,1.0


In [309]:
test_data.head()

,tweet,class,neg,pos,neu,compound,nouns,prons,verbs,adjs,...,num_symbols,stopword_count,emoji_count,hashtag,user,allcaps,total_length,num_words,num_unique_words,words_vs_unique
0,look ho look yes look no,1,0.000,0.505,0.495,0.8814,3,4,3,0,...,0,8,0,0,1,2,24.0,6.0,4.0,0.666667
1,lots some favorite not,0,0.000,0.223,0.777,0.5106,5,3,0,2,...,0,7,0,4,1,0,22.0,4.0,4.0,1.000000
2,wrong ppl nowadays was conducting sampling oth...,0,0.119,0.000,0.881,-0.4767,8,0,6,4,...,1,8,0,0,0,0,94.0,14.0,14.0,1.000000
3,lakers trash,1,0.000,0.000,1.000,0.0000,2,0,1,1,...,0,1,0,0,1,0,12.0,2.0,2.0,1.000000
4,will playing rf simpson nov,0,0.000,0.160,0.840,0.2714,5,0,2,1,...,0,4,0,2,1,0,27.0,5.0,5.0,1.000000


In [321]:
frames = [train_data,test_data]
df = pd.concat(frames,ignore_index=True)


In [328]:
test_data.to_csv("../dataset/test_cleaned_data")

In [324]:
df.dropna(inplace=True)

In [333]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

In [334]:
pipe = Pipeline(
        [('select', SelectFromModel(LogisticRegression(class_weight='balanced',
                                                  penalty="l1", C=0.01))),
        ('model', LogisticRegression(class_weight='balanced',penalty='l2'))])

In [451]:
Columns=train_data.columns[2:-1]

In [452]:
columns

Index(['neg', 'pos', 'neu', 'compound', 'nouns', 'prons', 'verbs', 'adjs',
       'advs', 'num_exclamation_marks', 'num_question_marks', 'num_symbols',
       'stopword_count', 'emoji_count', 'hashtag', 'user', 'allcaps',
       'total_length', 'num_words', 'num_unique_words', 'words_vs_unique'],
      dtype='object')

In [341]:
%run ../src/features/vectorization

In [505]:
train_vec,test_vec = ngram_vectorize(TfidfVectorizer,train_data["tweet"],train_data["class"],test_data["tweet"],top_k=10000,NGRAM_RANGE=(1,1))

In [527]:
train_vec_char,test_vec_char = ngram_vectorize(TfidfVectorizer,train_data["tweet"],train_data["class"],test_data["tweet"],top_k=20000,TOKEN_MODE='char',NGRAM_RANGE=(2,6))

In [431]:
%run ../src/classifier/NBSVM

In [518]:
model = NbSvmClassifier(C=10)

In [530]:
predict = model.predict(hstack((test_vec,test_vec_char)).tocsr())

In [574]:
df3 = hstack((test_vec,test_vec_char)).tocsr()

In [531]:
metrics.f1_score(test_data['class'],predict)

0.9304790605840174

In [575]:
df3.shape

(11866, 30000)

In [576]:
from scipy.sparse import csr_matrix, hstack


In [569]:
#frame = [train_vec.toarray(),train_data[Columns]]
df2 = hstack((train_vec,train_vec_char)).tocsr()

In [529]:
model.fit(df2,train_data["class"])

NbSvmClassifier(C=10, dual=False, n_jobs=1)

In [483]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [544]:
scores = []
classifier = LogisticRegression(C=10, solver='sag')
cv_score = np.mean(cross_val_score(classifier, df2, train_data["class"] ,cv=10, scoring='f1'))
scores.append(cv_score)
print('CV score for class {} is {}'.format(class_name, cv_score))

   

CV score for class 0 is 0.931595865109229


In [545]:
classifier.fit(df2, train_data["class"])
print('Total CV score is {}'.format(np.mean(scores)))

Total CV score is 0.931595865109229


In [570]:
model = LogisticRegression(solver='sag')
sfm = SelectFromModel(model, threshold=0.2)
train_sparse_matrix = sfm.fit_transform(df2, train_data['class'])
test_sparse_matrix = sfm.transform(df3)
d_train = lgb.Dataset(train_sparse_matrix, label=train_data["class"])
d_valid = lgb.Dataset(test_sparse_matrix, label=test_data["class"])
watchlist = [d_train, d_valid]
params = {'learning_rate': 0.2,
             'application': 'binary',
              'num_leaves': 31,
              'verbosity': -1,
              'metric': 'f1',
              'data_random_seed': 2,
              'bagging_fraction': 0.8,
              'feature_fraction': 0.6,
              'nthread': 4,
              'lambda_l1': 1,
          'lambda_l2': 1}
model1 = lgb.train(params,
                      train_set=d_train,
                      verbose_eval=10)
    


In [552]:
import lightgbm as lgb



In [557]:
?lgb.train

In [579]:
p = model.predict(test_sparse_matrix)

In [590]:
metrics.f1_score(test_data['class'],p>0.37)

0.9298892988929889

In [584]:
p

array([0.31527056, 0.02066055, 0.03974663, ..., 0.03376895, 0.0233928 ,
       0.98772505])